# TensorFlow 2.0

In [3]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline

## Hyperparameter Tunning

In [4]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [5]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [6]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [7]:
train_paths = glob('dataset/cifar/train/*.png')[:1000]
test_paths = glob('dataset/cifar/test/*.png')[:1000]

In [8]:
len(train_paths)

1000

In [9]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [10]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [11]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [12]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [13]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [16]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Checkpoint
- 모델이 학습할 때 도중에 weight를 저장해서 다음에 학습할 때 저장된 weight로 학습을 이어갈 수 있도록
- 게임으로 치면 save하는 것
- 중간에 실수로 학습을 꺼버리면 아까우니까 저장하는거지

In [17]:
save_path = 'checkpoints' # 저장경로

In [21]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# save_best_only=True: val_accuracy를 계속 모니터링하면서 accuracy가 올라가면 저장하고, 올라가지않으면 저장 안 함!
# monitor='val_loss'로 한다고하면 높아져야는 게 아니고 낮아져야하니까 mode='min'

## Training

In [22]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[checkpoint]
)

Epoch 1/20
30/31 [============================>.] - ETA: 0s - loss: 2.3149 - accuracy: 0.0962
Epoch 00001: val_accuracy improved from -inf to 0.11290, saving model to checkpoints
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: checkpoints\assets
31/31 [==============================] - 42s 1s/step - loss: 2.3143 - accuracy: 0.0961 - val_loss: 2.2862 - val_accuracy: 0.1129
Epoch 2/20
30/31 [============================>.] - ETA: 0s - loss: 2.2516 - accuracy: 0.1604
Epoch 00002: val_accuracy improved from 0.11290 to 0.17641, saving model to checkpoints
INFO:tensorflow:Assets written to: checkpoints\assets
31/31 [==============================] - 37s 1s/step - loss: 2.2479 - accuracy: 0.1673 - val_loss: 2.2076 - val_accuracy: 0.1764
Epoch 3/20
30/31 [============================>.] - ETA: 0s - loss: 2.1673 - accuracy: 0.1820
Epoch 00003: val_accuracy improved from 0.17641 to 0.22581, saving model to checkpoints
INFO:tenso

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup